In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
from collections import defaultdict


In [ ]:
%config Completer.use_jedi = False

In [ ]:
# refractory period 8s, 16 steps.

In [ ]:
CHANNEL_HL = 26 # BL

In [ ]:
class ChannelEnvironment:
    def __init__(self, food_coords=[0], enable_food_time=5, disable_food_time=100, refractory_period = 16):
        self.refractory_period = refractory_period
        
        self.food_w = np.pi/CHANNEL_HL/2. # half-width of the food., 2xfood_w = BL, same as step size.
        self.food = np.array(food_coords)
        self.food_enabled = np.array([False]*len(food_coords)) # all food locations disabled
        
        self.schedule = defaultdict(dict)
        #self.schedule = {5: dict(0=True), 20: dict(0=False)} # default schedule
        self.schedule[enable_food_time]={0:True}
        self.schedule[disable_food_time]={0:False}
        
        self.fly_on_food = None
        self.last_t = 0
        
        self.food_log = {"t":[], 0:[]}
        
    def env_state_update(self, t): # when the food is active
        self.last_t = t
        if t in self.schedule.keys():
            actions = self.schedule[t]
            for foodid, state in actions.items():
                self.set_enabled_food(foodid, state)
            del self.schedule[t]
            
        for ifood in range(len(self.food)):
            self.food_log["t"].append(self.last_t)
            self.food_log[ifood].append(self.food_enabled[ifood])
            
    def update(self, fly_coord, t):
        self.env_state_update(t)
        food_locs, food_indices = self.get_enabled_food_locations()
        for food_index,food_loc in zip(food_indices, food_locs):
             print("update:", t, food_loc, fly_coord, 
                   "fly-food:",np.abs(fly_coord - food_loc), 
                   "mod2pi:", np.abs(fly_coord - food_loc)%(2* np.pi))
             if np.abs(fly_coord - food_loc)%(2* np.pi) <= self.food_w:
                self.fly_on_food=food_index
                return self.fly_on_food
        self.fly_on_food=None
        return self.fly_on_food
        
    def get_enabled_food_locations(self):
        return self.food[self.food_enabled], self.food_enabled.nonzero()[0] #locations, indices
    
    def set_enabled_food(self, food_i, enabled_value):
        self.food_enabled[food_i] = enabled_value
        print(f"t={self.last_t}: Food #{food_i} [{self.food[food_i]}] := {enabled_value}")
        
    def enable_food(self, food_i=0):
        self.set_enabled_food(food_i, True)
        
    def disable_food(self, food_i=0, refractory=False):
        self.set_enabled_food(food_i, False)
        if refractory:
            for tbetween in range(self.last_t, self.last_t+self.refractory_period+1):
                if tbetween in self.schedule and not self.schedule[tbetween].get(food_i,True):
                    return                
            self.schedule[self.last_t+self.refractory_period][food_i]= True
    
    def print_current_state(self):
        print(np.vstack([self.food, self.food_enabled]))
    
    

In [ ]:
foods = np.array([1,2,3])
enabled = np.array([True,False, True])
foods[enabled]

In [ ]:
print(np.vstack([foods, enabled]))

In [ ]:
indices = enabled.nonzero()[0]

In [ ]:
phi_step = np.pi / CHANNEL_HL # 26 body length opposite side - figS3A
# 1 step 0.5 s, 1 BL

class MyFly:
    def __init__(self):
        self.eating_time = 10
        
        self.environment = ChannelEnvironment()
        self.t = 0
        
        # mind
        self.state='walking' # or eating
        self.mode='GS' # or LS
        self.direction=1 # or -1
        self.last_state=None
        self.integrator_x=0
        self.integrator_y=0
        
        self.current_run = 0
        
        # env
        self.coord_x=0
        self.coord_y=0
        self.coord_phi= - 8* phi_step
        
        # log
        self.phi_log=[self.coord_phi] # to save history
        self.t_log=[0]
        self.eat_log=[False]
        self.direction_log=[self.direction]
        
    def log(self, eating=False):
        self.t_log.append(self.t)
        self.phi_log.append(self.coord_phi)
        self.eat_log.append(eating)
        self.direction_log.append(self.direction)

    def make_step(self, direction):
        self.t+=1
        d_angle = phi_step*direction
        self.coord_phi += d_angle
        self.coord_x = np.cos(self.coord_phi)
        self.coord_y = np.sin(self.coord_phi)
        dx = np.cos(d_angle)
        dy = np.sin(d_angle)
        self.integrator_x+=dx
        self.integrator_y+=dy
        self.current_run += np.sqrt(dx**2+dy**2)
        
        self.log()
        
        am_on_food = self.environment.update(self.coord_phi, self.t)
        if am_on_food is not None:
            self.on_food(am_on_food)
    
    def plot_angle_history(self, ax=None):
        if ax is None:
            ax=plt.gca()
        ax.plot(self.t_log, self.phi_log, '.-')

    def plot_trajectory(self, ax=None):
        if ax is None:
            ax=plt.gca()
        
        angles = np.array(self.phi_log)
        xx = np.cos(angles)
        yy = np.sin(angles)
        ax.plot(xx, yy)
        ax.axis('equal')
    
    def on_food(self, food_index):
        print(self.t, " - fly on food!")
        self.mode = 'LS' # enter the local search mode

        # remember the eating state
        self.last_state = 'eating'

        # disable food source for refractory period
        self.environment.disable_food(food_index, refractory=True)
        
        # stay on food location for a while
        self.eat()
        
        self.zero_integrator()
        self.choose_run_length()
        
        
    def eat(self):
        # do nothing for self.eating time (only update the environment based on current time)
        for t in range(self.eating_time):
            self.t+=1
            # update log at every step
            self.log(eating=True)
#             self.t_log.append(self.t)
#             self.phi_log.append(self.coord_phi)
            # update environment
            print('fly eating', self.t)
            self.environment.update(self.coord_phi, self.t)

    def choose_run_length(self):
        # if just was on food
        if self.last_state=='eating':
            self.run_length = np.random.normal(RL_mean, RL_std) + self.current_run
        elif self.last_state == 'reversal':
            self.run_length = np.abs(self.current_run + np.random.normal(dRL_mean, dRL_std))
        print(f"Prev: {self.last_state}, current run: {self.current_run}, RL:{self.run_length}" )
            
    def zero_integrator(self):
        self.integrator_x = 0
        self.integrator_y = 0
        self.current_run = 0
        
    def start_walking(self, Tlim = 500):
        while self.mode == "GS":
            self.make_step(self.direction)
        if self.mode == 'LS':
            while self.t < Tlim:
                print(f"{self.t} | run: {self.current_run}")
                self.make_step(self.direction)
                if self.current_run >=self.run_length:
                    self.reversal()
    def reversal(self):
        self.direction *= -1        
        self.choose_run_length()
        self.zero_integrator()
        self.last_state = 'reversal'
        
    def get_df(self):
        df = pd.DataFrame(dict(t=fly.t_log, angle=fly.phi_log, 
                                 eating=fly.eat_log, direction=fly.direction_log))
        run_num = df.direction.diff().abs()/2
        df["run_num"] = run_num.cumsum()
        df.loc[0, "run_num"] = 0
        df.run_num = df.run_num - df[df.eating].iloc[-1].run_num - 1
        return df
    

In [ ]:
RL_mean = 4.125
RL_std = 2.625
dRL_mean = 0.03125
dRL_std = 1.875
np.random.normal(RL_mean, RL_std)

In [ ]:
f = MyFly()

In [ ]:
for i in range(3):
    f.make_step(1)
for i in range(30):
    f.make_step(-1)

In [ ]:
fig, ax = plt.subplots() 
f.plot_angle_history(ax)
plt.plot(f.environment.food_log["t"], f.environment.food_log[0])

In [ ]:
f.plot_trajectory()

In [ ]:
fly = MyFly()

In [ ]:
fly.start_walking(Tlim=200)

In [ ]:
fly.plot_angle_history()
plt.plot(fly.environment.food_log["t"], fly.environment.food_log[0])

In [ ]:
df = fly.get_df()

In [ ]:
run_num = df.direction.diff().abs()/2

In [ ]:
df["run_num"] = run_num.cumsum()

In [ ]:
df.loc[0, "run_num"] = 0

In [ ]:
df[df.eating].iloc[-1]

In [ ]:
#df.loc[0, "run_num"] = 0
df.run_num = df.run_num - df[df.eating].iloc[-1].run_num - 1

In [ ]:
for rn in [-1,0,1]:
    print(df[df.run_num ==rn].eating.unique())

In [ ]:
df.run_num.unique()

In [ ]:
run_info = df.groupby("run_num").aggregate({'angle':['first', 'last']})

In [ ]:
run_info.columns = ["_".join(x) for x in run_info.columns.ravel()]


In [ ]:
run_info

In [ ]:
run_info["run_midpoint"] = (run_info.angle_last + run_info.angle_first)/2

In [ ]:
run_info.run_midpoint

In [ ]:
df = pd.read_csv("fr_simulations.csv")

In [ ]:
df.head()

In [ ]:
run_info = df.groupby(["flyid","run_num"]).aggregate({'angle':['first', 'last']}).reset_index()
run_info.columns = ["_".join(x) for x in run_info.columns.ravel()]


In [ ]:
run_info['run_midpoint']=(run_info.angle_last + run_info.angle_first)/2

In [ ]:
h=plt.hist(run_info[run_info["run_num_"]<0].run_midpoint,density=True, bins=50)

In [ ]:
abins = np.linspace(-np.pi, np.pi,100)

In [ ]:
h=plt.hist(run_info[run_info["run_num_"]<0].run_midpoint,density=True, bins=abins)
h=plt.hist(run_info[run_info["run_num_"]>=0].run_midpoint,density=True, bins=abins)

In [ ]:
df.columns

In [ ]:

df = pd.read_csv("data/pheromone_simulations_4.csv")

In [ ]:
df.head()

In [ ]:
df.last_food_index.unique()

In [ ]:
df.last_food_coord.unique()

In [ ]:
# df.flyid.unique()

In [ ]:
i=0
for flyid, data in df.groupby("flyid"):
    if i>2: 
        break    
    plt.figure(figsize=(15,4))
    plt.plot(data.t, data.angle, color='k')
    plt.plot(data[data.eating].t, data[data.eating].angle, '.', color='red')
    plt.plot(data[data.smelling].t,data[data.smelling].angle , '.', color='cyan')
    plt.axhline(data.last_food_coord.iloc[0], ls='--')
    plt.axhline(data.last_food_coord.iloc[0]+2*np.pi, ls='--')
    plt.axhline(data.last_food_coord.iloc[0]-2*np.pi, ls='--')
    plt.axvline(600)
    plt.xlim(400,800)
    i+=1


In [ ]:
df['next_angle'] = df.angle.shift(-1)

In [ ]:
df['prev_angle'] = df.angle.shift(1) # need to do it in groups!!!

In [ ]:
df.loc[43:49]

In [ ]:
df.prev_angle.iloc[0] = df.prev_angle.iloc[1]

In [ ]:
df.head()

In [ ]:
run_info = df.groupby(["flyid","run_num"]).aggregate({'angle':['first', 'last'], 'last_food_index':'first', 'direction':'first'}).reset_index()
run_info.columns = ["_".join(x) for x in run_info.columns.ravel()]
run_info['run_midpoint']=(run_info.angle_last + run_info.angle_first)/2

In [ ]:
run_info.head()

In [ ]:
run_info.rename(columns={"run_num_": "run_num", "flyid_": "flyid",
                         "last_food_index_first":"last_food_index",
                        "direction_first":"direction"}, inplace=True)

In [ ]:
run_info.head()

In [ ]:
def angle_minuspitopi(angle):
    return (angle + np.pi) % (2 * np.pi) - np.pi

In [ ]:
run_info['theta_midpoint'] = run_info.run_midpoint.apply(lambda angle: angle_minuspitopi(angle))

In [ ]:
h=plt.hist(run_info[run_info["run_num"]<0].run_midpoint,density=True, bins=50)

In [ ]:
h=plt.hist(run_info[run_info["run_num"]<0].theta_midpoint,density=True, bins=50)

In [ ]:
h=plt.hist((run_info[run_info["run_num"]>=0].run_midpoint + np.pi) % (2*np.pi)-np.pi,density=True, bins=50)

In [ ]:
h=plt.hist(run_info[run_info["run_num"]>=0].run_midpoint,density=True, bins=50)

In [ ]:
import seaborn as sns

In [ ]:
run_info.columns


In [ ]:
post_runs = run_info[run_info['run_num']>=0]

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(data=post_runs, x='run_midpoint', hue='last_food_index', element='step', fill=False)
# plt.xlim(-np.pi, np.pi)

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(data=post_runs, x='theta_midpoint', hue='last_food_index', element='step', fill=False, bins=36)
# plt.xlim(-np.pi, np.pi)

In [ ]:
ax = plt.subplot(111, polar=True)
h=ax.hist(post_runs[post_runs.last_food_index_first==0].run_midpoint, bins=50)

In [ ]:
ax = plt.subplot(111, polar=True)
h=ax.hist(post_runs[post_runs.last_food_index_first==1].run_midpoint, bins=100)

In [ ]:
ax = plt.subplot(111, polar=True)
h=ax.hist(post_runs[post_runs.last_food_index_first==2].run_midpoint, bins=100)

In [ ]:
# run_info.rename(columns={"run_num_": "run_num", "flyid_": "flyid","last_food_index_first":"last_food_index"}, inplace=True)

In [ ]:
run_info.head()

In [ ]:
df.columns

In [ ]:
food_coords = df.last_food_coord.unique()

In [ ]:
def angle_in_range(angle, angle_from, angle_to):
    afrom = angle_minuspitopi(angle_from)    
    ato = angle_minuspitopi(angle_to)
#     print(afrom, ato)
    a = angle_minuspitopi(angle)
    if ato >= afrom:
#         print("ato>afrom")
        return (a < ato) and (a > afrom)
    if ato < afrom:
#         print(f"ato<afrom; a={a}")
        a = angle_minuspitopi(a+np.pi)        
#         print(a)
#         print("a<afrom", a<afrom)
#         print("a>ato", a>ato)
        return (a<afrom) and (a > ato)

        

In [ ]:
def angle_in_range2(angle, angle_from, angle_to):
    if abs(angle_to - angle_from)>=2*np.pi:
        return True
    afrom = angle_minuspitopi(angle_from)    
    ato = angle_minuspitopi(angle_to)    
    a = angle_minuspitopi(angle)
    reversing = ((ato-afrom)*(angle_to - angle_from) < 0)
#     print("reversing", reversing)
    if not reversing:
#         print(afrom, ato)
#         print(a-ato, a-afrom)
        return (a - ato) * (a - afrom) < 0
    if reversing:
#         print(a-ato, a-afrom)
        a = angle_minuspitopi(a+np.pi)        
#         print(a-ato, a-afrom)
        return (a - ato) * (a - afrom) < 0


In [ ]:
angle_in_range(0, -np.pi/4, np.pi/4)

In [ ]:
angle_in_range2(0, -np.pi/4, np.pi/4)

In [ ]:
angle_in_range(0, np.pi/4, np.pi/2)

In [ ]:
angle_in_range(0, np.pi/4, np.pi/2)

In [ ]:
angle_in_range(np.pi, np.pi/4, np.pi+7*np.pi/8)

In [ ]:
angle_in_range2(np.pi, np.pi/4, np.pi+7*np.pi/8)

In [ ]:
angle_in_range(np.pi/4, -np.pi/4, 0)

In [ ]:
angle_in_range2(0, np.pi/4, -np.pi/4)

In [ ]:
angle_in_range(0, np.pi/4, -np.pi/4)

In [ ]:
for ifood, food_coord in enumerate(food_coords):
    col = f"contains_{ifood}"
    run_info[col] = run_info.apply(lambda row: angle_in_range(food_coord, row.angle_first, row.angle_last), axis=1)

In [ ]:
run_info.head()

In [ ]:
run_info["contains_food"] = run_info.contains_0.astype(int) + run_info.contains_1.astype(int) + run_info.contains_2.astype(int)

In [ ]:
run_info.contains_food.value_counts()

In [ ]:
ap_runs = run_info[run_info.run_num<0]

In [ ]:
ap_runs.shape

In [ ]:
last_two_runs = ap_runs[ap_runs.run_num>=-3]

In [ ]:
last_two_runs.head(10)

In [ ]:
last_two_runs.groupby("run_num").contains_food.value_counts()

In [ ]:
good_ones3 = ap_runs[(ap_runs.run_num==-3) & (ap_runs.contains_food==3)]

In [ ]:
flies3=good_ones3.flyid.unique()

In [ ]:
flies3

In [ ]:
flies2=good_ones.flyid.unique()

In [ ]:
set(flies2).intersection({1,22,345})

In [ ]:
set(flies3).intersection(set(flies2))

In [ ]:
good_ones = ap_runs[(ap_runs.run_num==-2) & (ap_runs.contains_food==3)]

In [ ]:
good_ones

In [ ]:
good_ones.last_food_index.value_counts()

In [ ]:
good_flies = good_ones.flyid

In [ ]:
good_flies

In [ ]:
example_flyids = good_ones.groupby("last_food_index").flyid.first()

In [ ]:
example_flyids

In [ ]:
for fly in example_flyids:
    data = df[df.flyid==fly]
    plt.figure(figsize=(15,4))
    plt.plot(data.t, data.angle, color='k')
    plt.plot(data[data.eating].t, data[data.eating].angle, '.', color='red')
    plt.plot(data[data.smelling].t,data[data.smelling].angle , '.', color='cyan')
    plt.axhline(data.last_food_coord.iloc[0], ls='--')
    plt.axhline(data.last_food_coord.iloc[0]+2*np.pi, ls='--')
    plt.axhline(data.last_food_coord.iloc[0]-2*np.pi, ls='--')
    plt.axvline(600)
    plt.xlim(400,1000)    
    plt.title(str(fly))

In [ ]:
ap_runs[(ap_runs.flyid==22)&(ap_runs.run_num>=-3)]

In [ ]:
a1 = 1.087474
a2 = -0.845813
a=0


In [ ]:
angle_in_range?

In [ ]:
angle_in_range(a, a1, a2)

In [ ]:
# plt.figure(figsize=(15,8))
sns.histplot(data=post_runs[post_runs.flyid.isin(good_flies)], x='theta_midpoint', hue='last_food_index', 
             element='step', fill=False, stat='density', common_norm=False, bins=20)
# plt.xlim(-np.pi, np.pi)